In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
# spark = SparkSession.builder.appName('walmart-data').master('spark://spark-master:7077').getOrCreate()
spark = SparkSession.builder.appName('walmart-data').master('local').getOrCreate()

In [33]:
product_schema = 'product_id INT, product_name STRING, aisle_id INT, department_id INT'

order_product_schema = 'order_id INT, product_id INT, add_to_cart_order INT, reordered INT'

In [29]:
products_df = spark.read.option("header", True).schema(product_schema).csv('./datasets/products.csv')
products_df.show(5)

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|Chocolate Sandwic...|      61|           19|
|         2|    All-Seasons Salt|     104|           13|
|         3|Robust Golden Uns...|      94|            7|
|         4|Smart Ones Classi...|      38|            1|
|         5|Green Chile Anyti...|       5|           13|
+----------+--------------------+--------+-------------+


In [30]:
products_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)


In [31]:
products_df = products_df.select('product_id')  # Items set
products_df.show(5)

+----------+
|product_id|
+----------+
|         1|
|         2|
|         3|
|         4|
|         5|
+----------+


In [32]:
products_df.count()

49688

In [55]:
order_products_df = spark.read.option("header", True).schema(order_product_schema).csv('./datasets/order_products__prior.csv')
order_products_df.show(5)

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       2|     33120|                1|        1|
|       2|     28985|                2|        1|
|       2|      9327|                3|        0|
|       2|     45918|                4|        1|
|       2|     30035|                5|        0|
+--------+----------+-----------------+---------+


In [42]:
order_products_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)


In [56]:
order_products_df = order_products_df.drop("add_to_cart_order", "reordered")
order_products_df.show(5)

+--------+----------+
|order_id|product_id|
+--------+----------+
|       2|     33120|
|       2|     28985|
|       2|      9327|
|       2|     45918|
|       2|     30035|
+--------+----------+


In [44]:
order_products_df.count()

150792

In [57]:
order_products_df = order_products_df.groupBy('order_id').agg(f.collect_set('product_id').alias('items_transactions'))

In [58]:
order_products_df.show(truncate=False)  # Transactions

+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|order_id|items_transactions                                                                                                                                                                        |
+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|2       |[17794, 1819, 40141, 9327, 28985, 45918, 30035, 33120, 43668]                                                                                                                             |
|3       |[32665, 33754, 17461, 17704, 21903, 17668, 24838, 46667]                                                                                                                                  |
|4       |

In [50]:
products_df.withColumnRenamed('frequency', f.array_contains(order_products_df.items_transactions, products_df.product_id)).show()

PySparkTypeError: [NOT_ITERABLE] Column is not iterable.

In [ ]:
min_support = 0.3
transactions_qty = order_products_df.count()

products_df = products_df.withColumnRenamed('frequency', )